In [1]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder
tqdm.pandas()
pd.options.display.max_columns = None

In [2]:
fifa15 =pd.read_csv('../data/FIFA/players_15.csv')
fifa16 =pd.read_csv('../data/FIFA/players_16.csv')
fifa17 =pd.read_csv('../data/FIFA/players_17.csv')
fifa18 =pd.read_csv('../data/FIFA/players_18.csv')
fifa19 =pd.read_csv('../data/FIFA/players_19.csv')
fifa20 =pd.read_csv('../data/FIFA/players_20.csv')
fifa21 =pd.read_csv('../data/FIFA/players_21.csv')
fifa22 =pd.read_csv('../data/FIFA/players_22.csv')

/Users/pierreadda/opt/anaconda3/envs/foot_env/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (104) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/pierreadda/opt/anaconda3/envs/foot_env/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (81,82,83,84,85) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/pierreadda/opt/anaconda3/envs/foot_env/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (25,108) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
clubs = pd.read_csv('../data/games/clubs.csv', index_col = 0)
leagues = pd.read_csv('../data/games/leagues.csv', index_col = 0)
games = pd.read_csv('../data/games/games.csv', index_col = 0,
                   dtype = {'game_id' : np.int32, 'season' : np.int16, 'home_club_id' : np.int32, 'home_club_id' : np.int32,
                           'home_club_goals' : np.int8, 'away_club_goals' : np.int8})

players = pd.read_csv('../data/games/players.csv', index_col = 0,
                     dtype = {'player_id' : np.int32, 'player_club_id' : np.int16})

appearance = pd.read_csv('../data/games/appearance.csv', index_col = 0)
corr = pd.read_csv('../data/table_correspondance/table_correspondance_finale.csv', index_col = 0)
player_IDs = corr.player_id.unique()


In [4]:
def game_id_toint(gameid):
    try:
        res = int(gameid)
    except:
        res = 'error : '+str(gameid)
    return res
appearance.loc[:,'game_id'] = appearance.game_id.apply(game_id_toint)

#some rows have nan for game_id, we don't keep them.

appearance = appearance.drop(appearance.loc[appearance.game_id.apply(type) != int,:].index).reset_index(drop = True)

# Game_id selection
<b> We keep games that have more than 9 players _in each team_ that we can locate in our fifa dataset 

In [5]:
# In appearance, we drop lines with players that have not been found in FIFA

appearance_ID = appearance.loc[appearance.player_id.isin(player_IDs),:].copy()
print( ' Lines with missing players:' ,round((appearance.shape[0]-appearance_ID.shape[0])/appearance.shape[0]*100,2), '%')

 Lines with missing players: 10.81 %


In [6]:
# With a grouby - count, we select teams having more than X players

nb_players = 6 # we want games where we know 9 or more players
group_app = appearance_ID.groupby(['game_id','player_club_id']).count().reset_index()
print(group_app.shape[0])
group_app = group_app.loc[group_app.player_id >= nb_players,:].reset_index(drop = True)
print(group_app.shape[0])

# Now that we deleted lines with missing players in appearance, and teams with less than X players:
# some games have only one team left, we want to remove these games.
# we create group_app_ID with game_id and a boolean describing wether the game have the 2 teams left or not:

group_app_ID = group_app.groupby('game_id')['player_club_id'].unique().reset_index()
group_app_ID.loc[:,'player_club_id'] = group_app_ID.player_club_id.apply(len)>1
print(group_app_ID.loc[group_app_ID.player_club_id,:].shape , group_app_ID.shape)

#good_games is a list of games ids that we can use for model training!
good_games = group_app_ID.loc[group_app_ID.player_club_id,'game_id'].values

72655
66262
(30700, 2) (35562, 2)


In [7]:
#Now let's create appearance_select with only the games we selected
appearance_select = appearance.loc[appearance.game_id.isin(good_games),:].reset_index(drop = True).copy()
appearance_select.to_csv('../data/training_data/appearance_select6.csv')

### Home or Away?

In [8]:
appearance_select = \
pd.read_csv('../data/training_data/appearance_select6.csv', index_col = 0, \
            dtype = {'game_id' : np.int32, 'player_id' : np.int32,'player_club_id' : np.int16, 'minutes_played' : np.int8,\
                    'yellow_cards' : np.int8,'red_cards' : np.int8,'goals' : np.int8,'assists' : np.int8 })


In [9]:
def HomeOrAway(x):
    if games.loc[games.game_id == x['game_id'], 'home_club_id'].values[0] == x['player_club_id']:
        return 'Home'
    else:
        return 'Away'

In [11]:
appearance_select['HomeOrAway'] = appearance_select.progress_apply(lambda x: HomeOrAway(x) , axis = 1)

100%|██████████| 860694/860694 [04:32<00:00, 3152.98it/s]


In [12]:
appearance_select.to_csv('../data/training_data/appearance_select6.csv')

# Ratio of player_appearance_time/game_full_time
<b> This will help us handle players that have been substituted.  
    In our Dataset we have no way to know which players are starters or off-benchs.  
    We will use this ratio to weight the players stats.

In [13]:
appearance_select = \
pd.read_csv('../data/training_data/appearance_select6.csv', index_col = 0, \
            dtype = {'game_id' : np.int32, 'player_id' : np.int32,'player_club_id' : np.int16, 'minutes_played' : np.int8,\
                    'yellow_cards' : np.int8,'red_cards' : np.int8,'goals' : np.int8,'assists' : np.int8 })

In [14]:
max_time = appearance_select.groupby('game_id')['minutes_played'].max().reset_index().astype({'game_id':np.int32})
    

In [15]:
max_time.minutes_played.unique()

array([ 90, 120,  68,  45,  86], dtype=int8)

In [16]:
#Let's remove games that didn't last for at least 90 minutes (anomalies)

bad_games = max_time.loc[max_time.minutes_played.isin([68,  45,  86]),'game_id'].values
appearance_select = appearance_select.drop(appearance_select.loc[appearance_select.game_id.isin(bad_games),:].index).reset_index(drop = True)

In [17]:
# Add game full time in a new column (either 90 or 120)

max_time = appearance_select.groupby('game_id')['minutes_played'].max().reset_index().astype({'game_id':np.int32})
games_120 = max_time.loc[max_time.minutes_played == 120,'game_id'].values
appearance_select['max_time'] = np.int8(90)
appearance_select.loc[appearance_select.game_id.isin(games_120),'max_time'] = np.int8(120)

In [18]:
# Adds the time ratio of how much of the game a player has played.

appearance_select['time_ratio'] = np.float16(appearance_select.minutes_played / appearance_select.max_time)
appearance_select.head(2)

,player_id,game_id,appearance_id,competition_id,player_club_id,goals,assists,minutes_played,yellow_cards,red_cards,HomeOrAway,max_time,time_ratio
0,52453,2483937,2483937_52453,RU1,28095,0,0,90,0,0,Away,90,1.0
1,67064,2479929,2479929_67064,RU1,28095,0,0,90,0,0,Away,90,1.0


In [19]:
appearance_select.to_csv('../data/training_data/appearance_select6.csv')

# Players position
<b> We will attribute either Attack, Goalkeeper, Midfield or Defender to each player.  

In [20]:
appearance_select = \
pd.read_csv('../data/training_data/appearance_select6.csv', index_col = 0, \
            dtype = {'game_id' : np.int32, 'player_id' : np.int32,'player_club_id' : np.int16, 'minutes_played' : np.int8,\
                    'yellow_cards' : np.int8,'red_cards' : np.int8,'goals' : np.int8,'assists' : np.int8,
                    'max_time' : np.int16,'time_ratio' : np.float16})

In [21]:
#Ther is ONE player with no position nor sub_position, let's remedy this (We checked the position in Fifa)

players.loc[players.player_id == 290801,'position'] = 'Defender'
players.loc[players.player_id == 290801,'sub_position'] = 'Centre-Back'

In [22]:
# Add players position (attack, midfield, defense or goal keeper)
t  = players.loc[players.position == '0','sub_position']
players.loc[t.index,'position'] = t

In [23]:
players.drop(columns =
             ['name','pretty_name','country_of_birth','country_of_citizenship','foot','height_in_cm','market_value_in_gbp' ,'highest_market_value_in_gbp' ,'url'],
            inplace = True)
players.head(1)


,player_id,current_club_id,date_of_birth,position,sub_position
0,38790,28095,1985-06-24,Attack,Centre-Forward


In [24]:
players.position = players.position.apply(lambda x: x.lower())
players.position.unique()

array(['attack', 'goalkeeper', 'midfield', 'defender'], dtype=object)

In [25]:
appearance_select['position'] = appearance_select.player_id.progress_apply(lambda x: players.loc[players.player_id == x,'position'].values[0])


100%|██████████| 860619/860619 [03:46<00:00, 3792.77it/s]


In [26]:
###################################

appearance_select.to_csv('../data/training_data/appearance_select6.csv')

In [27]:
appearance_select = \
pd.read_csv('../data/training_data/appearance_select6.csv', index_col = 0, \
            dtype = {'game_id' : np.int32, 'player_id' : np.int32,'player_club_id' : np.int16, 'minutes_played' : np.int8,\
                    'yellow_cards' : np.int8,'red_cards' : np.int8,'goals' : np.int8,'assists' : np.int8,
                    'max_time' : np.int16,'time_ratio' : np.float16})

In [28]:
OHE = OneHotEncoder(sparse = False)
position_OHE = OHE.fit_transform(np.asarray(appearance_select.position).reshape(-1, 1))
position_OHE = pd.DataFrame(position_OHE)

In [29]:
print(OHE.get_feature_names())
position_OHE.head(1)

['x0_attack' 'x0_defender' 'x0_goalkeeper' 'x0_midfield']


/Users/pierreadda/opt/anaconda3/envs/foot_env/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,0,1,2,3
0,1.0,0.0,0.0,0.0


In [30]:
position_OHE.rename(columns={0:'attack', 1:'defense', 2:'GK', 3:'midfield'},inplace=True)
position_OHE.head(1)

,attack,defense,GK,midfield
0,1.0,0.0,0.0,0.0


In [24]:
#appearance_select.drop(columns=['attack','defense','GK','midfield'], inplace = True)

In [31]:
appearance_select = pd.concat([appearance_select, position_OHE], axis=1)
train_0 = appearance_select.loc[:,['player_id','game_id','player_club_id','time_ratio','attack','defense','GK','midfield']].copy()
train_0.head(1)

,player_id,game_id,player_club_id,time_ratio,attack,defense,GK,midfield
0,52453,2483937,28095,1.0,1.0,0.0,0.0,0.0


In [32]:

train_0.loc[:,['attack' ,'defense' ,'GK' ,'midfield']] =\
train_0.loc[:,['attack' ,'defense' ,'GK' ,'midfield']].multiply(train_0.time_ratio, axis = 0)
train_0.head(10)

,player_id,game_id,player_club_id,time_ratio,attack,defense,GK,midfield
0,52453,2483937,28095,1.000000,1.000000,0.0,0.0,0.0
1,67064,2479929,28095,1.000000,1.000000,0.0,0.0,0.0
2,67064,2483937,28095,1.000000,1.000000,0.0,0.0,0.0
3,67064,2484582,28095,0.611328,0.611328,0.0,0.0,0.0
4,67064,2485965,28095,1.000000,1.000000,0.0,0.0,0.0
5,67064,2487345,28095,1.000000,1.000000,0.0,0.0,0.0
6,67064,2498464,28095,1.000000,1.000000,0.0,0.0,0.0
7,67064,2500260,28095,0.211060,0.211060,0.0,0.0,0.0
8,67064,2506733,28095,1.000000,1.000000,0.0,0.0,0.0
9,67064,2508366,28095,0.022217,0.022217,0.0,0.0,0.0


In [33]:
train_0 = train_0.groupby(['game_id','player_club_id'])[['attack','defense','GK','midfield']].sum().reset_index()
train_0.to_csv('train1_6.csv')

# Which FIFA?
<b> In which fifa we should search for the player? 

In [34]:
# Add game season:

appearance_select['season'] = appearance_select.game_id.apply(
    lambda x: games.loc[games.game_id == x,'season'].values[0]-2000
)


In [35]:
appearance_select.to_csv('../data/training_data/appearance_select6.csv')

In [36]:
appearance_select = \
pd.read_csv('../data/training_data/appearance_select6.csv', index_col = 0, \
            dtype = {'game_id' : np.int32, 'player_id' : np.int32,'player_club_id' : np.int16, 'minutes_played' : np.int8,\
                    'yellow_cards' : np.int8,'red_cards' : np.int8,'goals' : np.int8,'assists' : np.int8,
                    'max_time' : np.int16,'time_ratio' : np.float16, 'season' : np.int8})

In [37]:
# in known_players, add the FIFA in which the player can be found
known_players = players.loc[players.player_id.isin(player_IDs),['player_id' ,'current_club_id','position' ,'sub_position']]
known_players['which_fifa'] = known_players.player_id.apply(lambda x:
                                                             corr.loc[corr.player_id == x,'which_fifa'].values
                                                            )
known_players.head()

,player_id,current_club_id,position,sub_position,which_fifa
0,38790,28095,attack,Centre-Forward,[15]
1,106539,28095,attack,Right Winger,[15]
2,164389,2288,attack,Centre-Forward,"[15, 16, 17]"
3,45247,1162,goalkeeper,Goalkeeper,"[15, 16, 17, 18, 19, 20]"
4,97205,28095,attack,Right Winger,[15]


In [38]:
#in appearance_select, select players that we know:
appearance_known_players = appearance_select.loc[appearance_select.player_id.isin(player_IDs),:].copy()
appearance_known_players['which_fifa']=None

In [40]:
# There are 2757 players in known players that do not appear in appearance_select
# We will drop them
print(known_players.shape[0],end = ' ')
index = known_players.loc[~known_players.player_id.isin(appearance_known_players.player_id.unique()),:].index
known_players = known_players.drop(index).reset_index(drop = True)
print(known_players.shape[0],end = ' ')

16369 13742 

In [42]:
#test before apply:
x = appearance_known_players.iloc[0,:]
%time known_players.loc[known_players.player_id == x['player_id'],'which_fifa'].values[0]


CPU times: user 1.11 ms, sys: 342 µs, total: 1.45 ms
Wall time: 1.19 ms


array([15, 16])

In [43]:
#add which_fifa column

appearance_known_players.loc[:,'which_fifa'] = \
appearance_known_players.player_id.progress_apply(lambda x:
                                         known_players.loc[known_players.player_id == x,'which_fifa'].values[0]
                                        )

100%|██████████| 804484/804484 [03:35<00:00, 3734.31it/s]


In [44]:
appearance_known_players.to_csv('../data/training_data/appearance_known_players6.csv')

In [45]:
#test
#####PROBLEME METTRE LES SOUSTRACTION EN VALEURS ABSOLUES!!!!!!!
x = appearance_known_players.iloc[400200,:]
t = (np.argwhere(abs(x['season']-x['which_fifa']) == (abs(x['season']-x['which_fifa'])).min())).squeeze()
x['season'], x['which_fifa'],x['which_fifa'][t], 

(17, array([15, 16, 17, 18, 19, 20, 21]), 17)

In [46]:
appearance_known_players.loc[:,'which_fifa'] = \
appearance_known_players.progress_apply(lambda x:
                               x['which_fifa'][(np.argwhere(abs(x['season']-x['which_fifa']) == (abs(x['season']-x['which_fifa'])).min())).squeeze()]
                               ,axis = 1
                              )


100%|██████████| 804484/804484 [01:13<00:00, 10933.86it/s]


In [47]:
#Certains joueurs ont 2 which_fifa possible. On prendra systématiquement le plus récent.
mask = appearance_known_players.which_fifa.apply(type)!=np.int64
appearance_known_players.loc[mask,'which_fifa'] = appearance_known_players.loc[mask,'which_fifa'].apply(lambda x: x[-1])

In [48]:
appearance_known_players.to_csv('../data/training_data/appearance_known_players6.csv')